In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from google.cloud import bigquery

In [2]:
%%bigquery df
SELECT * FROM `agussman-bigquery-demo.census_dataset.input_view`

In [3]:
df.head()

,age,workclass,native_country,marital_status,education_num,occupation,race,hours_per_week,income_bracket,dataframe
0,39,Private,United-States,Married-civ-spouse,5,Other-service,Black,34,<=50K,training
1,72,Private,United-States,Married-civ-spouse,5,Exec-managerial,Asian-Pac-Islander,48,>50K,training
2,45,Private,United-States,Married-civ-spouse,5,Machine-op-inspct,White,40,>50K,training
3,31,Private,United-States,Married-civ-spouse,5,Exec-managerial,White,40,<=50K,evaluation
4,55,Private,United-States,Married-civ-spouse,5,Tech-support,White,23,<=50K,training


In [4]:
df.iloc[0].income_bracket

' <=50K'

In [5]:
df.loc[df.income_bracket == " <=50K", 'label'] = 1
df.loc[df.income_bracket != " <=50K", 'label'] = 0

In [6]:
df.head()

,age,workclass,native_country,marital_status,education_num,occupation,race,hours_per_week,income_bracket,dataframe,label
0,39,Private,United-States,Married-civ-spouse,5,Other-service,Black,34,<=50K,training,1.0
1,72,Private,United-States,Married-civ-spouse,5,Exec-managerial,Asian-Pac-Islander,48,>50K,training,0.0
2,45,Private,United-States,Married-civ-spouse,5,Machine-op-inspct,White,40,>50K,training,0.0
3,31,Private,United-States,Married-civ-spouse,5,Exec-managerial,White,40,<=50K,evaluation,1.0
4,55,Private,United-States,Married-civ-spouse,5,Tech-support,White,23,<=50K,training,1.0


In [7]:
df.dtypes

age                 int64
workclass          object
native_country     object
marital_status     object
education_num       int64
occupation         object
race               object
hours_per_week      int64
income_bracket     object
dataframe          object
label             float64
dtype: object

In [8]:
for col in ['workclass', 'native_country', 'marital_status', 'occupation', 'race']:
    dest = f'{col}_cat'
    df[dest] = pd.Categorical(df[col]).codes

In [9]:
df.tail()

,age,workclass,native_country,marital_status,education_num,occupation,race,hours_per_week,income_bracket,dataframe,label,workclass_cat,native_country_cat,marital_status_cat,occupation_cat,race_cat
32556,20,Private,United-States,Never-married,10,Other-service,Asian-Pac-Islander,30,<=50K,training,1.0,4,39,4,8,1
32557,48,Private,Cambodia,Married-civ-spouse,10,Craft-repair,Asian-Pac-Islander,40,>50K,training,0.0,4,1,2,3,1
32558,22,Local-gov,United-States,Never-married,10,Protective-serv,White,43,<=50K,training,1.0,2,39,4,11,4
32559,20,Private,United-States,Never-married,10,Adm-clerical,White,20,<=50K,training,1.0,4,39,4,1,4
32560,43,State-gov,United-States,Divorced,10,Adm-clerical,White,40,<=50K,training,1.0,7,39,0,1,4


In [10]:
# dump these
for col in ['workclass', 'native_country', 'marital_status', 'occupation', 'race', 'education_num', 'income_bracket']:
    df.pop(col)

In [11]:
training_df = df.loc[df.dataframe == 'training']
eval_df = df.loc[df.dataframe == 'evaluation']

In [12]:
training_df.head()

,age,hours_per_week,dataframe,label,workclass_cat,native_country_cat,marital_status_cat,occupation_cat,race_cat
0,39,34,training,1.0,4,39,2,8,2
1,72,48,training,0.0,4,39,2,4,1
2,45,40,training,0.0,4,39,2,7,4
4,55,23,training,1.0,4,39,2,13,4
5,30,40,training,1.0,4,32,2,7,4


In [13]:
training_df.pop('dataframe')
eval_df.pop('dataframe')

3        evaluation
23       evaluation
28       evaluation
61       evaluation
76       evaluation
            ...    
32470    evaluation
32471    evaluation
32488    evaluation
32527    evaluation
32555    evaluation
Name: dataframe, Length: 3226, dtype: object

In [14]:
t_target = training_df.pop('label')
e_target = eval_df.pop('label')

In [15]:
t_dataset = tf.data.Dataset.from_tensor_slices((training_df.values, t_target.values))
e_dataset = tf.data.Dataset.from_tensor_slices((eval_df.values, e_target.values))

In [16]:
t_dataset

<TensorSliceDataset shapes: ((7,), ()), types: (tf.int64, tf.float64)>

In [17]:
for feat, targ in e_dataset.take(5):
  print ('Features: {}, Target: {}'.format(feat, targ))

Features: [31 40  4 39  2  4  4], Target: 1.0
Features: [48 40  4 39  2  1  4], Target: 1.0
Features: [40 32  4 39  2  8  2], Target: 1.0
Features: [19 40  0 25  2  0  1], Target: 1.0
Features: [29 14  4 30  2  6  1], Target: 1.0


In [18]:
train_dataset = t_dataset.shuffle(len(training_df)).batch(1)
eval_dataset = e_dataset.shuffle(len(eval_df)).batch(1)

In [19]:
# From https://www.tensorflow.org/tutorials/load_data/pandas_dataframe
# Not using this one at the moment
if False:
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(10, activation='relu'),
        tf.keras.layers.Dense(1)
      ])

    model.compile(optimizer='adam',
                    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                    metrics=['accuracy'])

In [20]:
# modded to remove feature_layer from https://www.tensorflow.org/io/tutorials/bigquery
# Using this model
Dense = tf.keras.layers.Dense
model = tf.keras.Sequential(
  [
#     feature_layer,
      Dense(100, activation=tf.nn.relu, kernel_initializer='uniform'),
      Dense(75, activation=tf.nn.relu),
      Dense(50, activation=tf.nn.relu),
      Dense(25, activation=tf.nn.relu),
      Dense(1, activation=tf.nn.sigmoid)
  ])

# Compile Keras model
model.compile(
    loss='binary_crossentropy', 
    metrics=['accuracy'])

In [21]:
model.fit(train_dataset, epochs=15)

Train for 26193 steps
Epoch 1/15
26193/26193 [==============================] - 82s 3ms/step - loss: 0.6450 - accuracy: 0.7570
Epoch 2/15
26193/26193 [==============================] - 83s 3ms/step - loss: 0.6959 - accuracy: 0.7575
Epoch 3/15
26193/26193 [==============================] - 82s 3ms/step - loss: 0.7146 - accuracy: 0.7553
Epoch 4/15
26193/26193 [==============================] - 84s 3ms/step - loss: 0.7306 - accuracy: 0.7536
Epoch 5/15
26193/26193 [==============================] - 81s 3ms/step - loss: 0.7107 - accuracy: 0.7538
Epoch 6/15
26193/26193 [==============================] - 81s 3ms/step - loss: 0.6906 - accuracy: 0.7536
Epoch 7/15
26193/26193 [==============================] - 81s 3ms/step - loss: 0.6699 - accuracy: 0.7559
Epoch 8/15
26193/26193 [==============================] - 83s 3ms/step - loss: 0.7089 - accuracy: 0.7553
Epoch 9/15
26193/26193 [==============================] - 81s 3ms/step - loss: 0.7280 - accuracy: 0.7544
Epoch 10/15
26193/26193 [========

In [22]:
loss, accuracy = model.evaluate(eval_dataset)
print("Accuracy", accuracy)

3226/3226 [==============================] - 6s 2ms/step - loss: 0.5007 - accuracy: 0.7719
Accuracy 0.7718537
